In [ ]:
# 모델 저장 함수
import torch 

torch.save(
    model,
    path
)

#### 모델 불러오기

In [6]:
import inspect # 파이썬 내부 클래스나 함수의 소스 코드 확인 할 수 있음

from torch import nn

print(inspect.getsource(nn.Module))

class Module:
    r"""Base class for all neural network modules.

    Your models should also subclass this class.

    Modules can also contain other Modules, allowing to nest them in
    a tree structure. You can assign the submodules as regular attributes::

        import torch.nn as nn
        import torch.nn.functional as F

        class Model(nn.Module):
            def __init__(self):
                super().__init__()
                self.conv1 = nn.Conv2d(1, 20, 5)
                self.conv2 = nn.Conv2d(20, 20, 5)

            def forward(self, x):
                x = F.relu(self.conv1(x))
                return F.relu(self.conv2(x))

    Submodules assigned in this way will be registered, and will have their
    parameters converted too when you call :meth:`to`, etc.

    .. note::
        As per the example above, an ``__init__()`` call to the parent class
        must be made before assignment on the child.

    :ivar training: Boolean represents whether this module is in

In [14]:
import torch
from torch import nn

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2,1)

    def forward(self, x):
        x = self.layer(x)
        return x
    
device = torch.device("mps")
model = torch.load("./models/model.pt", map_location=device) # 저장된 클래스 이름에 해당하는 실제 클래스 정의를 현재 환경에서 찾기 때문에 위에 CustomModel 선언
print(model)

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)
tensor([[  1.5358],
        [ 69.2628],
        [357.2008]], device='mps:0')


In [13]:
# 모델 구조를 모르겠다면 모델 구조를 출력할 수 있음
# 변수의 명칭까지 동일한 형태로 구현
import torch
from torch import nn

class CustomModel(nn.Module):
    pass

device = torch.device("mps")
model = torch.load("./models/model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


#### 모델 상태 저장/불러오기|

In [15]:
# 모델 상태만 저장
import torch

torch.save(
    model.state_dict(),
    "./models/model_state_dict.pt"
)

In [17]:
# 즉 가중치와 편향이 저장되어 있음
print(model.state_dict())

OrderedDict([('layer.weight', tensor([[ 3.1058, -1.7030]], device='mps:0')), ('layer.bias', tensor([0.1329], device='mps:0'))])


In [1]:
# 모델 상태 불러오기
import torch
from torch import nn

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2,1)

    def forward(self, x):
        x = self.layer(x)
        return x
    
device = torch.device('mps')
model = CustomModel().to(device)

model_state_dict = torch.load('./models/model_state_dict.pt', map_location=device)
model.load_state_dict(model_state_dict)

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)

#### 체크포인트 저장/불러오기

In [5]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)
    
    def forward(self, x):
        x = self.layer(x)
        return x
    
train_dataset = CustomDataset("./dataset/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = torch.device('mps')
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

checkpoint = 1
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = x.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel", # 모델 식별자
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f'./models/checkpoint-{checkpoint}.pt',
        )

        checkpoint += 1

/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_py3.8/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([128, 2])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [6]:
### 체크포인트 불러오기

import torch
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        x = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length
    
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2,1)

    def forward(self, x):
        x = self.layer(x)
        return x
    
checkpoint = torch.load("./models/checkpoint-6.pt")
model.load_state_dict(checkpoint["model_state_dict"]) # 위에 선언한 model_state_dict 선언
optimizer.load_state_dict(checkpoint["optimizer_state_dict"]) # 위에 선언한 optimizer_state_dict 선언
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

for epoch in range(checkpoint_epoch + 1, 10000): # 체크 포인트 시점 점 부터
    cost = 0.0

    for x,y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output= model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f'Epoch: {epoch+1:4d}, Model : {list(model.parameters())}, Cost: {cost:.3f}')

CustomModel 체크포인트-6
Epoch: 7000, Model : [Parameter containing:
tensor([[ 3.1043, -1.7002]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2413], device='mps:0', requires_grad=True)], Cost: 0.095
Epoch: 8000, Model : [Parameter containing:
tensor([[ 3.1040, -1.7029]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2628], device='mps:0', requires_grad=True)], Cost: 0.102
Epoch: 9000, Model : [Parameter containing:
tensor([[ 3.1037, -1.7032]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.2825], device='mps:0', requires_grad=True)], Cost: 0.100
Epoch: 10000, Model : [Parameter containing:
tensor([[ 3.1034, -1.7031]], device='mps:0', requires_grad=True), Parameter containing:
tensor([0.3004], device='mps:0', requires_grad=True)], Cost: 0.093
